## Lazysetup
 1. Provides a method for adding to an object dynamically.
 2. Object attributes do not need to be pre-defined.
 3. This would also allow for attributes to take on default values.

In [1]:
print('something')

something


In [ ]:
# lazysetup.py
from modules.gate_logger import Logger

class lazysetup(object):
    def __init__(self):
        self.logger = Logger(__name__)
        self.maint_cli = 'maint_cli container'
        self.ff_gui = 'Firefox container'

    def __getattr__(self, name):
        """
        Function is only called if the attribute did not already exist
        """
        value = None
        self.logger.general('Initializing attribute:{0}'.format(name))
        if name == 'wls_client':
            value = 'Wireless client container'
        setattr(self, name, value)
        return value


class forcelazy(object):
    def __init__(self):
        self.logger = Logger(__name__)

    def __getattribute__(self, name):
        """
        Function is always called
        """
        print('\nAttribute:{0} was called'.format(name))
        try:
            return object.__getattribute__(self, name)
        except AttributeError:
            value = None
            setattr(self, name, value)
            return value

## test_mytestmodule.py


In [ ]:
#from lazysetup import lazysetup, forcelazy
import pytest

### The setup fixture is adding:
 1. DDC container
 2. Wireless Client Container
 3. PLaceholder attribute
#### A fixture is used so that the code after yield (cleanup) will be executed after every test run.

In [ ]:
@pytest.fixture()
def setup_fixture():
    """
    Setup for the module
    """

    env = lazysetup()
    env.logger.general("\nIn setup fixture")
    env.ddc = "ddc setup"
    env.wls_client
    env.placeholder

    yield env

    print "\nCleaning up env"

## The setup_too fixture
Is using a get_attribute method that will run every time the attribute is referenced. This may be useful for containers where a check is run to ensure the container is running and responsive. 

In [ ]:
@pytest.fixture()
def setup_too():
    """
    Forced attribute execution
    """
    env2 = forcelazy()
    print env2.ff_gui
    env2.ff_gui = 'My Gui'
    print env2.ff_gui

    yield env2

    print "\nCleaning up other env"

def test_something(setup_fixture):
    """
    Test using env lazy setup
    """
    env = setup_fixture
    print "Testing my lazy setup"
    print "Updating placeholder"
    env.placeholder = 'Assigned value'

    print "CLI = {0}".format(env.maint_cli)
    print "GUI = {0}".format(env.ff_gui)
    print "DDC = {0}".format(env.ddc)
    print "WLS Client = {0}".format(env.wls_client)
    print "Adding something in the test"
    env.something = "Something new"

    print "Running test"
    try:
        raise Exception("Whoops")
    except:
        env.logger.critical("Something went wrong")
        env.logger.critical("My env is {}".format(env.__dict__))

def test_else(setup_too):
    """
    Test using env lazy setup
    """
    env2 = setup_too
    print "Testing my lazy setup"

    print "GUI = {0}".format(env2.ff_gui)
    print "Adding something in the test"
    env2.something = "Something new"

    print "Running test"
    try:
        raise Exception("Whoops")
    except:
        env2.logger.critical("Something went wrong")
        env2.logger.critical("My env is {}".format(env2.__dict__))
        
